In [ ]:
import pathlib

import geopandas as gpd
import pandas as pd
import panel as pn
import pydeck as pdk

pn.extension("deckgl")

In [ ]:
# See data/README.md to download this file
DATA_ROOT_PATH = pathlib.Path().absolute().parent / "data"

In [ ]:
community_next_globus_info = gpd.read_parquet(
    DATA_ROOT_PATH / "processed" / "german_communities.parquet",
)
community_next_globus_info["distance_km"] = community_next_globus_info["distance"].apply(
    lambda d: f"{(d / 1000):.2f}",
)
community_next_globus_info

In [ ]:
globus_positions = pd.read_csv(DATA_ROOT_PATH / "processed" / "globus_info.csv", sep=";")
globus_positions["geometry"] = list(
    zip(globus_positions["lon"], globus_positions["lat"], strict=False),
)
globus_positions["lon_fmt"] = globus_positions["lon"].apply(lambda d: f"{d:.6f}")
globus_positions["lat_fmt"] = globus_positions["lat"].apply(lambda d: f"{d:.6f}")
globus_positions

In [ ]:
extended_community_info = community_next_globus_info.join(
    globus_positions[["name", "location"]],
    on="globus_row",
)
extended_community_info

In [ ]:
INITIAL_VIEW_STATE = pdk.ViewState(latitude=51.3, longitude=10, zoom=6, max_zoom=16, bearing=0)

geojson = pdk.Layer(
    "GeoJsonLayer",
    extended_community_info,
    opacity=0.5,
    stroked=True,
    filled=True,
    get_fill_color=[
        "(distance_km / 200) * 255",
        "255 - ((distance_km / 200) * 255)",
        "(distance_km / 200) * 255",
    ],
    get_line_color=[0, 0, 0],
    auto_highlight=True,
    pickable=True,
)
point_layer = pdk.Layer(
    "ScatterplotLayer",
    globus_positions,
    opacity=0.8,
    stroked=True,
    filled=True,
    get_fill_color=[255, 255, 255],
    get_position="geometry",
    radius_scale=1,
    radius_min_pixels=5,
    radius_max_pixels=10,
    get_radius=10,
    line_width_min_pixels=1,
    auto_highlight=True,
    pickable=True,
)

community_tooltip = {"text": "Community: {community}\nNext: {name}\nDistance: {distance_km} km"}
globus_tooltip = {"text": "Name: {name}\nLocation: {location}\nCoordinates: ({lon_fmt}, {lat_fmt})"}

tooltips = {
    geojson.id: community_tooltip,
    point_layer.id: globus_tooltip,
}

pdk_deck = pdk.Deck(
    layers=[geojson, point_layer],
    initial_view_state=INITIAL_VIEW_STATE,
    tooltip=tooltips,
    map_provider=None,
)

globus_map = pn.pane.DeckGL(pdk_deck, sizing_mode="stretch_width", min_height=1200)
globus_map

In [ ]:
overview_text = pn.pane.Markdown(
    """
This interactive map provides a visual representation of the proximity of \
Globus supermarkets to various communities across Germany. Each element of the map has been \
thoughtfully designed to enhance user understanding of the distances involved.

The white dots on the map indicate the precise locations of Globus supermarkets, \
serving as reference points for users to identify their proximity to these stores. \
Surrounding these points are color-coded community polygons, which are shaded according \
to the distance to the nearest Globus supermarket. A vibrant green hue signifies that a \
community is situated close to a Globus store, while a deeper purple indicates a greater distance.

As you hover over each community polygon, a tooltip appears, displaying key information: \
the name of the community along with the calculated distance to the nearest Globus supermarket. \
This distance is measured "as the crow flies," representing the shortest path from the geographic \
center of the community to the nearest store location.
""",
    sizing_mode="stretch_both",
)
overview_text

In [ ]:
data_text = pn.pane.Markdown(
    """
This map is based on two data sources.

The geographic data is derived from the dataset titled [Verwaltungsgebiete 1:5 000 000, Stand \
31.12. (VG5000 31.12.)](https://gdz.bkg.bund.de/index.php/default/digitale-geodaten/\
verwaltungsgebiete/verwaltungsgebiete-1-5-000-000-stand-31-12-vg5000-12-31.html), \
raised and copyright by © GeoBasis-DE / BKG (2024), \
which is freely available for use.

Market information for Globus was extracted from the [Globus Market location overview]\
(https://www.globus.de/maerkte.php). I then utilized \
[Nominatim](https://nominatim.openstreetmap.org/) from OpenStreetMap to \
manually search for the precise location\\ of the markets. The OSM data is shared under the \
[Open Data Commons Open Database License (ODbL)](https://opendatacommons.org/licenses/odbl/).
""",
    sizing_mode="stretch_both",
)
data_text

In [ ]:
template = pn.template.ReactTemplate(title="Globus Map", prevent_collision=True)

text_content = pn.Tabs(
    ("Overview", overview_text),
    ("Data", data_text),
    styles={"font-size": "18px", "--bokeh-font-size": "16px"},
)
template.main[:8, :7] = globus_map
template.main[:8, 7:11] = text_content
template.servable()